In [1]:
!pip install -r ../../requirements.txt

  Cloning https://github.com/ryan112358/private-pgm.git (to revision 5b9126295c110b741e5426ddbff419ea1e60e788) to /tmp/pip-install-hpexqwll/private-pgm_aa8d511759a64cca9e4e4574d5df15e1
  Running command git clone --filter=blob:none --quiet https://github.com/ryan112358/private-pgm.git /tmp/pip-install-hpexqwll/private-pgm_aa8d511759a64cca9e4e4574d5df15e1
  Running command git rev-parse -q --verify 'sha^5b9126295c110b741e5426ddbff419ea1e60e788'
  Running command git fetch -q https://github.com/ryan112358/private-pgm.git 5b9126295c110b741e5426ddbff419ea1e60e788
  Resolved https://github.com/ryan112358/private-pgm.git to commit 5b9126295c110b741e5426ddbff419ea1e60e788
  Preparing metadata (setup.py) ... done
  Using cached anonymeter-1.0.0-py3-none-any.whl.metadata (10 kB)
  Using cached antlr4-python3-runtime-4.9.3.tar.gz (117 kB)
  Preparing metadata (setup.py) ... done
  Using cached boto3-1.34.58-py3-none-any.whl.metadata (6.6 kB)
  Using cached botocore-1.34.58-py3-none-any.whl.metad

In [2]:
import pandas as pd
import numpy as np

In [3]:
# %cd /Users/alex/PETsARD

import os
import sys

path_petsard = os.path.dirname(os.path.dirname(os.getcwd()))
print(path_petsard)
sys.path.append(path_petsard)

/home/ec2-user/SageMaker/PETsARD


In [4]:
from PETsARD import (
    Loader,
    Processor,
    Synthesizer
)


load = Loader(
    filepath='benchmark://adult-income',
    na_values={k: '?' for k in [
        'workclass',
        'occupation',
        'native-country'
    ]}
)
load.load()

ModuleNotFoundError: No module named 'sdv'

In [ ]:
load.data = load.data.loc[:1000, :]

In [ ]:
import pprint


def issue332(
        load: Loader,
        synthesizing_method: str,
        scaler_inhibit: bool = False
):
    pp = pprint.PrettyPrinter(depth=2)

    proc_discretizing = Processor(
        metadata=load.metadata,
    )

    if scaler_inhibit:
        print("Preproc config of Scaler before update as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])
        proc_discretizing.update_config(
            {'scaler': {
                col: None for col in load.data.columns
            }}
        )
        print("Preproc config of Scaler before after as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])

    proc_discretizing.fit(
        data=load.data,
        sequence=[
            'missing',
            'outlier',
            'scaler',
            'discretizing'
        ]
    )
    preproc_discretizing_data = proc_discretizing.transform(
        data=load.data
    )
    print("Preproc data as ...")
    print(preproc_discretizing_data.head(1))


    syn = Synthesizer(
        method=synthesizing_method,
        epsilon=10.0,
    )
    syn.create(data=preproc_discretizing_data)
    syn.fit_sample()
    print("Sync data as ...")
    print(syn.data_syn.head(1))
    postproc_discretizing_data = proc_discretizing.inverse_transform(
        data=syn.data_syn
    )
    print(postproc_discretizing_data.head(1))

In [ ]:
import pprint


def issue332_gan(
        load: Loader,
        synthesizing_method: str,
        scaler_inhibit: bool = False,
        outlier_inhibit: bool = False
):
    pp = pprint.PrettyPrinter(depth=2)

    proc_discretizing = Processor(
        metadata=load.metadata,
    )

    if scaler_inhibit:
        print("Preproc config of Scaler before update as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])
        proc_discretizing.update_config(
            {'scaler': {
                col: None for col in load.data.columns
            }}
        )
        print("Preproc config of Scaler before after as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])

    if outlier_inhibit:
        print("Preproc config of Outlier before update as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])
        proc_discretizing.update_config(
            {'outlier': {
                col: None for col in load.data.columns
            }}
        )
        print("Preproc config of Outlier before after as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])


    proc_discretizing.fit(
        data=load.data,
        sequence=[
            'missing',
            'outlier',
            'encoder',
            'scaler'
        ]
    )
    preproc_discretizing_data = proc_discretizing.transform(
        data=load.data
    )
    print("Preproc data as ...")
    print(preproc_discretizing_data.head(1))


    syn = Synthesizer(
        method=synthesizing_method,
        epsilon=10.0,
    )
    syn.create(data=preproc_discretizing_data)
    syn.fit_sample()
    print("Sync data as ...")
    print(syn.data_syn.head(1))
    postproc_discretizing_data = proc_discretizing.inverse_transform(
        data=syn.data_syn
    )
    print(postproc_discretizing_data.head(1))

In [ ]:
issue332(
    load=load,
    synthesizing_method='smartnoise-aim'
)

上述執行會出現 `RuntimeError: all elements of input should be between 0 and 1`，原因是 input 中含有 NA 值，來自於前處理過程中將離群值去除掉，導致 `capital-gain`, `capital-loss` 兩個欄位的值皆相同，在 `smartnoise` 中進行 `MinMaxTransformer` 時會讓運算結果為 NA （分母為 0）。因此針對 adult 資料集，將其中的 `outlier` 前處理拿掉，就可以正常運行了。

In [ ]:
issue332(
    load=load,
    synthesizing_method='smartnoise-mst'
)

In [ ]:
issue332(
    load=load,
    synthesizing_method='smartnoise-pacsynth'
)

In [ ]:
issue332_gan(
        load = load,
        synthesizing_method = 'smartnoise-dpctgan',
        outlier_inhibit = True
)

In [ ]:
issue332_gan(
        load = load,
        synthesizing_method = 'smartnoise-patectgan',
        outlier_inhibit = True
)